
# Panoptic segmentation on NPM3D (Colab-ready)

Ce notebook prépare l'environnement recommandé dans le README, télécharge les données NPM3D depuis Google Drive et montre comment lancer l'entraînement/l'évaluation tout en offrant la possibilité de brancher un algorithme de clustering personnalisé sur l'embedding 5D.



## Configuration rapide

Utilisez les interrupteurs ci-dessous pour activer les étapes coûteuses. Par défaut, le notebook se contente d'initialiser les utilitaires et d'exécuter un test rapide du plugin de clustering de façon à pouvoir tourner même sans GPU. Activez les options lorsque vous travaillez sur une machine disposant de la configuration complète.


In [ ]:

from pathlib import Path
import os
import sys

repo_root = Path.cwd()
if (repo_root / 'pyproject.toml').exists() and (repo_root / 'torch_points3d').exists():
    pass
elif (repo_root / 'PanopticSegForLargeScalePointCloud').exists():
    repo_root = repo_root / 'PanopticSegForLargeScalePointCloud'
    os.chdir(repo_root)
else:
    raise RuntimeError("Impossible de localiser la racine du dépôt. Exécutez le notebook depuis le répertoire du projet.")

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print(f"Repository root: {repo_root}")


In [ ]:

import json
from datetime import datetime
from pathlib import Path

INSTALL_ENV = False
DOWNLOAD_DATA = False
RUN_TRAINING = False
RUN_EVAL = False
RUN_STATS = False
USE_CUSTOM_CLUSTER = True

ENV_NAME = 'treeins_env_local'
CONDA_ROOT = Path('/opt/conda')

DATA_ROOT = Path(os.environ.get('NPM3D_DATA_ROOT', repo_root / 'data'))
OUTPUT_ROOT = Path(os.environ.get('NPM3D_OUTPUT_ROOT', repo_root / 'outputs'))
CHECKPOINT_DIR = Path(os.environ['NPM3D_CHECKPOINT']) if os.environ.get('NPM3D_CHECKPOINT') else None
EVAL_RESULTS_DIR = Path(os.environ['NPM3D_EVAL_DIR']) if os.environ.get('NPM3D_EVAL_DIR') else None

CUSTOM_CLUSTER_SPEC = {
    'target': 'scripts.custom_clustering.dbscan_plugin:cluster_embeddings',
    'kwargs': {'eps': 0.45, 'min_samples': 15},
    'default_type': 0,
}

JOB_NAME = f"npm3d_run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

env_flags = {
    'INSTALL_ENV': '1' if INSTALL_ENV else '0',
    'DOWNLOAD_DATA': '1' if DOWNLOAD_DATA else '0',
    'RUN_TRAINING': '1' if RUN_TRAINING else '0',
    'RUN_EVAL': '1' if RUN_EVAL else '0',
    'RUN_STATS': '1' if RUN_STATS else '0',
    'CONDA_ENV_NAME': ENV_NAME,
    'CONDA_ROOT': str(CONDA_ROOT),
    'DATA_ROOT': str(DATA_ROOT),
    'OUTPUT_ROOT': str(OUTPUT_ROOT),
}
os.environ.update(env_flags)

custom_cluster_overrides = []
if USE_CUSTOM_CLUSTER and CUSTOM_CLUSTER_SPEC:
    target = CUSTOM_CLUSTER_SPEC.get('target')
    if target:
        custom_cluster_overrides.append('models.PointGroup-PAPER.cluster_type=custom')
        custom_cluster_overrides.append(f'models.PointGroup-PAPER.custom_clustering.target={target}')
    for key, value in (CUSTOM_CLUSTER_SPEC.get('kwargs') or {}).items():
        custom_cluster_overrides.append(
            f'models.PointGroup-PAPER.custom_clustering.kwargs.{key}={value}'
        )
    if 'default_type' in CUSTOM_CLUSTER_SPEC:
        custom_cluster_overrides.append(
            f"models.PointGroup-PAPER.custom_clustering.default_type={CUSTOM_CLUSTER_SPEC['default_type']}"
        )

print('Interrupteurs :')
flag_summary = {k: (v.lower() in {'1', 'true'}) for k, v in env_flags.items() if k in {'INSTALL_ENV', 'DOWNLOAD_DATA', 'RUN_TRAINING', 'RUN_EVAL', 'RUN_STATS'}}
print(json.dumps(flag_summary, indent=2))
print('Répertoires utilisés :')
print(json.dumps({'data_root': str(DATA_ROOT), 'output_root': str(OUTPUT_ROOT)}, indent=2))
print(f'Overrides clustering : {custom_cluster_overrides}')


In [ ]:

import subprocess
from typing import Sequence


def run_subprocess(cmd: Sequence[str], *, cwd=None, env=None, check=True):
    final_env = os.environ.copy()
    if env:
        final_env.update(env)
    printable = ' '.join(cmd)
    print(f'$ {printable}')
    subprocess.run(cmd, cwd=cwd, env=final_env, check=check)


def conda_run_prefix():
    conda_exe = CONDA_ROOT / 'bin' / 'conda'
    if not conda_exe.exists():
        raise FileNotFoundError(
            f"Conda introuvable dans {conda_exe}. Lancez l'installation de l'environnement au préalable."
        )
    return [str(conda_exe), 'run', '-n', ENV_NAME]


print('Utilitaires initialisés.')


## Installation de l'environnement (optionnelle)

In [ ]:

%%bash
set -euo pipefail

if [[ "${INSTALL_ENV:-0}" != "1" ]]; then
  echo "Installation ignorée car INSTALL_ENV=0."
  exit 0
fi

CONDA_ROOT="${CONDA_ROOT:-/opt/conda}"
ENV_NAME="${CONDA_ENV_NAME:-treeins_env_local}"

if [[ ! -d "${CONDA_ROOT}" ]]; then
  echo "Installation de Miniconda dans ${CONDA_ROOT}..."
  wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O Miniconda3.sh
  bash Miniconda3.sh -b -p "${CONDA_ROOT}"
else
  echo "Miniconda déjà présent dans ${CONDA_ROOT}."
fi

source "${CONDA_ROOT}/etc/profile.d/conda.sh"
conda config --set always_yes yes
conda update -n base conda
conda deactivate >/dev/null 2>&1 || true

if ! conda env list | grep -q "^${ENV_NAME} "; then
  echo "Création de l'environnement ${ENV_NAME}..."
  conda create -n "${ENV_NAME}" python=3.8
else
  echo "L'environnement ${ENV_NAME} existe déjà."
fi

conda activate "${ENV_NAME}"
conda install pytorch=1.9.0 torchvision=0.10.0 torchaudio=0.9.0 cudatoolkit=11.1 -c pytorch -c nvidia
pip install numpy==1.19.5
conda install openblas-devel -c anaconda
export CUDA_HOME=/usr/local/cuda-11
pip install -U git+https://github.com/NVIDIA/MinkowskiEngine -v --no-deps --install-option="--blas_include_dirs=${CONDA_PREFIX}/include" --install-option="--blas=openblas"
pip install torch-scatter==2.0.8 -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
pip install torch-sparse==0.6.12 -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
pip install torch-geometric==1.7.2
pip install -r requirements.txt
pip install numba==0.55.1
conda install -c conda-forge hdbscan==0.8.27
conda install numpy-base==1.19.2
pip install joblib==1.1.0
conda deactivate
echo "Environnement ${ENV_NAME} prêt."



## Téléchargement des données (optionnel)

Les fichiers sont hébergés sur Google Drive. Le code suivant utilise l'API `gdown` depuis Python pour éviter les soucis de quoting dans les cellules bash. Si le téléchargement échoue (erreur 403, quota), montez votre Drive (`from google.colab import drive`) et copiez manuellement les fichiers dans `data/npm3dfused/raw`.


In [ ]:

import subprocess
import sys
from pathlib import Path

if not DOWNLOAD_DATA:
    print('Téléchargement ignoré car DOWNLOAD_DATA=0.')
else:
    data_root = Path(os.environ.get('DATA_ROOT', repo_root / 'data'))
    raw_dir = data_root / 'npm3dfused' / 'raw'
    raw_dir.mkdir(parents=True, exist_ok=True)
    try:
        import gdown  # type: ignore
    except ModuleNotFoundError:
        subprocess.run([sys.executable, '-m', 'pip', 'install', 'gdown'], check=True)
        import gdown  # type: ignore

    folder_url = 'https://drive.google.com/drive/folders/1Tsb-sEFmjIk458RqqfIu9pq7PgSoAwbp?usp=sharing'
    print(f'Téléchargement des fichiers NPM3D dans {raw_dir}')
    gdown.download_folder(folder_url, quiet=False, use_cookies=False, output=str(raw_dir))


## Inspection du jeu de données

In [ ]:

raw_dir = Path(os.environ.get('DATA_ROOT', repo_root / 'data')) / 'npm3dfused' / 'raw'
if raw_dir.exists():
    train_files = sorted(raw_dir.glob('*_train.ply'))
    val_files = sorted(raw_dir.glob('*_val.ply'))
    test_files = sorted(raw_dir.glob('*_test.ply'))
    print(f"{len(train_files)} fichiers train / {len(val_files)} val / {len(test_files)} test trouvés dans {raw_dir}.")
else:
    train_files = val_files = test_files = []
    print(f'Pas de données trouvées dans {raw_dir}. Montez ou téléchargez le jeu de données.')
TEST_FILES = test_files


## Test du plugin de clustering personnalisé

In [ ]:

if USE_CUSTOM_CLUSTER and CUSTOM_CLUSTER_SPEC:
    print('Chargement du plugin...')
    try:
        from torch_points3d.utils.custom_cluster import resolve_custom_clusterer
        adapter = resolve_custom_clusterer(CUSTOM_CLUSTER_SPEC)
    except ModuleNotFoundError as exc:
        print(f"Dépendances manquantes pour le plugin ({exc}). Installez l'environnement complet pour le tester.")
    except Exception as exc:
        print(f"Échec du chargement du plugin: {exc}")
    else:
        try:
            import torch
        except ModuleNotFoundError:
            print('PyTorch non disponible dans ce runtime, test ignoré.')
        else:
            torch.manual_seed(0)
            embeddings = torch.randn(256, 5)
            embeddings[:128] += 2.0
            embeddings[128:] -= 2.0
            batch_ids = torch.zeros(256, dtype=torch.long)
            global_indices = torch.arange(256, dtype=torch.long)
            predicted_labels = torch.randint(0, 3, (256,), dtype=torch.long)
            dummy_semantics = torch.randn(256, 9)
            dummy_offsets = torch.zeros(256, 3)
            clusters, cluster_types = adapter(
                embeddings=embeddings,
                batch_ids=batch_ids,
                global_indices=global_indices,
                predicted_labels=predicted_labels,
                semantic_logits=dummy_semantics,
                positions=torch.randn(256, 3),
                offsets=dummy_offsets,
                raw_positions=torch.randn(256, 3),
                raw_embeddings=embeddings,
                raw_offsets=dummy_offsets,
                raw_semantic_logits=dummy_semantics,
                raw_batch_ids=batch_ids,
                ignore_labels=torch.tensor([-1, 0, 1], dtype=torch.long),
                bandwidth=None,
                metadata={'notebook': True},
            )
            print(f'Plugin OK : {len(clusters)} clusters (types {cluster_types.tolist()}).')
else:
    print('Plugin personnalisé désactivé.')


## Commande d'entraînement

In [ ]:

train_args = [
    'python',
    'train.py',
    'task=panoptic',
    'data=panoptic/npm3d-sparseconv_grid_012_R_16_cylinder_area1',
    'models=panoptic/area4_ablation_3heads_5',
    'model_name=PointGroup-PAPER',
    'training=7_area1',
    f'job_name={JOB_NAME}',
]
train_args.extend(custom_cluster_overrides)
try:
    conda_prefix_cmd = list(conda_run_prefix())
except FileNotFoundError:
    conda_prefix_cmd = None
if conda_prefix_cmd:
    display_command = ' '.join(conda_prefix_cmd + train_args)
else:
    display_command = ' '.join(train_args)
print('Commande :')
print(display_command)
if RUN_TRAINING:
    if not conda_prefix_cmd:
        raise FileNotFoundError("Environnement conda introuvable. Lancez la cellule d'installation.")
    run_subprocess(conda_prefix_cmd + train_args, cwd=repo_root)
else:
    print("Entraînement ignoré (RUN_TRAINING=False).")


## Commande d'évaluation

In [ ]:

import json

eval_args = [
    'python',
    'eval.py',
    'task=panoptic',
    'data=panoptic/npm3d-sparseconv_grid_012_R_16_cylinder_area1',
    'models=panoptic/area4_ablation_3heads_5',
    'model_name=PointGroup-PAPER',
    'training=7_area1',
]
if CHECKPOINT_DIR:
    eval_args.append(f'checkpoint_dir={CHECKPOINT_DIR}')
if TEST_FILES:
    eval_args.append(f'data.fold={json.dumps([str(p) for p in TEST_FILES])}')
eval_args.extend(custom_cluster_overrides)
try:
    conda_prefix_cmd = list(conda_run_prefix())
except FileNotFoundError:
    conda_prefix_cmd = None
if conda_prefix_cmd:
    display_command = ' '.join(conda_prefix_cmd + eval_args)
else:
    display_command = ' '.join(eval_args)
print('Commande :')
print(display_command)
if RUN_EVAL:
    if not CHECKPOINT_DIR:
        raise ValueError("Définissez CHECKPOINT_DIR avant de lancer l'évaluation.")
    if not TEST_FILES:
        raise ValueError("Aucun fichier test détecté. Téléchargez le jeu de données.")
    if not conda_prefix_cmd:
        raise FileNotFoundError("Environnement conda introuvable. Lancez la cellule d'installation.")
    run_subprocess(conda_prefix_cmd + eval_args, cwd=repo_root)
else:
    print("Évaluation ignorée (RUN_EVAL=False).")


## Statistiques finales

In [ ]:

if RUN_STATS:
    if not EVAL_RESULTS_DIR:
        raise ValueError("Définissez EVAL_RESULTS_DIR vers le dossier des sorties d'évaluation.")
    try:
        conda_prefix_cmd = list(conda_run_prefix())
    except FileNotFoundError:
        raise
    run_subprocess(
        conda_prefix_cmd + ['python', 'evaluation_stats_NPM3D.py'],
        cwd=repo_root,
        env={'TP3D_EVAL_RESULTS': str(EVAL_RESULTS_DIR)},
        check=False,
    )
    print("Adaptez evaluation_stats_NPM3D.py pour utiliser TP3D_EVAL_RESULTS ou modifiez-le via argparse selon vos besoins.")
else:
    print("Statistiques ignorées (RUN_STATS=False). Modifiez evaluation_stats_NPM3D.py ou utilisez vos propres scripts pour exploiter les sorties dans outputs/.")



## Étapes suivantes

1. Exécutez l'entraînement en activant `RUN_TRAINING`.
2. Mettez à jour `CHECKPOINT_DIR` avec le dossier de sortie généré par l'entraînement, puis passez `RUN_EVAL` à `True`.
3. Ajustez `evaluation_stats_NPM3D.py` pour pointer vers `EVAL_RESULTS_DIR` (ou adaptez le script pour lire l'argument d'environnement `TP3D_EVAL_RESULTS`) et activez `RUN_STATS` afin de comparer vos résultats à ceux de l'article.
